# Scripts for diverse analyses

This notebook applies the `Get_Job_Script.ipynb` to automatically generate the sbatch scripts to run in Yale's cluster. 

Here the scripts generated are to run:

1. PCA analysis
2. Detect missingness in plink files
3. Extract SNPs/Individuals using Plink
4. Run regenie burden MWE

## File paths on Yale cluster
- Genotype files exome data:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb28374_exomedata/exome_data_OCT2020`
- Genotype files in PLINK format:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/UKB_Caucasians_phenotypeindepqc120319_updated020720removedwithdrawnindiv`
- Genotype files in bgen format:
`SAY/dbgapstg/scratch/UKBiobank/genotype_files/ukb39554_imputeddataset/`
- Summary stats for imputed variants BOLT-LMM:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data`
- Summary stats for inputed variants FastGWA:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data`
- Phenotype files:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/pleiotropy_R01/phenotypesforanalysis/UKB_Caucasiansubset_cholesterolfields_adjbymedstatus_062420_foranalysis`
- Relationship file:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/unrelated_n307259/UKB_unrelatedcauc_phenotypes_asthmat2dbmiwaisthip_agesex_waisthipratio_040620`
- Other traits to be analyzed:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/pleiotropy_R01/phenotypesforanalysis/UKB_CAUC_lipidsforanalysis_apolipoproteinAandB,Hba1c_continuousandcategorical,egfrbyCKDEPI,serumcreatinine,UACR_inverseranknorm_110320`

In [2]:
# Common variables
UKBB_PATH=/gpfs/gibbs/pi/dewan/data/UKBiobank
USER_PATH=/home/dc2325/project
OUT_PATH=/home/dc2325/project/UKBB_GWAS_dev/output
tpl_file=$USER_PATH/UKBB_GWAS_dev/farnam.yml
#pca_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/pca_exomes
pca_dir=~/scratch60/pca
famFile=$UKBB_PATH/genotype_files/ukb28374_exomedata/exome_data_OCT2020/ukb23155_s200631.fam
# Use a subset of the exomed markers
genoFile=`echo $UKBB_PATH/genotype_files/ukb28374_exomedata/exome_data_OCT2020/ukb23155_c{1..22}_b0_v1.bed`
#Use the original bed files for the genotype array on regenie step1
bfile=$UKBB_PATH/genotype_files/pleiotropy_geneticfiles/UKB_originalgenotypefilesdownloaded083019/UKB_genotypedatadownloaded083019.bed
phenoFile=~/scratch60/pca/cache/ukb23155_s200631.non_white_white_outliers_11971ind.pheno
database=$UKBB_PATH/phenotype_files/pleiotropy_R01/ukb42495_updatedJune2020/ukb42495.tab
ethnia_prefix='non_white_white_outliers_11971ind'
select_ethnia=2001,3001,4001,2002,3002,4002,2003,3003,4003,2004,3004,2,3,4,5,6,-3,-1
# Container
container_lmm=$UKBB_PATH/lmm.sif
# Pipeline
pca_sos=$USER_PATH/UKBB_GWAS_dev/workflow/PCA.ipynb
# Name of bash script
#pca_sbatch=../output/$(date +"%Y-%m-%d")_pca_non_white.sbatch
pca_sbatch=../output/$(date +"%Y-%m-%d")_flashpca_non_white_whiteoutliers.sbatch
numThreads=1
job_size=1
#PCA variables
k=10
maxiter=0
topk=10
sigma=6
window=50
shift=5
r2=0.5

## PCA jobs

### 1. African ancestry

In [5]:
pca_dir=~/scratch60/pca/african_ancestry
ethnia_prefix='african_3690ind'
phenoFile=~/scratch60/pca/african_ancestry/cache/ukb23155_s200631.african_3690ind.pheno
pca_sbatch=$OUT_PATH/$(date +"%Y-%m-%d")_flashpca_african.sbatch
trait_name=ethnicity

pca_args="""flashpca
    --cwd $pca_dir 
    --genoFile $genoFile
    --famFile $famFile
    --database $database
    --ethnia_prefix $ethnia_prefix
    --select_ethnia $select_ethnia
    --phenoFile $phenoFile
    --k $k
    --maxiter $maxiter
    --topk $topk
    --sigma $sigma
    --window $window
    --shift $shift
    --r2 $r2
    --trait_name $trait_name
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $pca_sos \
    --to-script $pca_sbatch \
    --args "$pca_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   /home/dc2325/project/UKBB_GWAS_dev/output/2021-02-22_flashpca_african.sbatch
INFO: Workflow farnam (ID=wd99c56feacb4bc44) is executed successfully with 1 completed step.



### 2. Asian ancestry

In [7]:
pca_dir=~/scratch60/pca/asian_ancestry
ethnia_prefix='asian_4618ind'
phenoFile=~/scratch60/pca/asian_ancestry/cache/ukb23155_s200631.asian_4618ind.pheno
pca_sbatch=$OUT_PATH/$(date +"%Y-%m-%d")_flashpca_asian.sbatch
trait_name=ethnicity

pca_args="""flashpca
    --cwd $pca_dir 
    --genoFile $genoFile
    --famFile $famFile
    --database $database
    --ethnia_prefix $ethnia_prefix
    --select_ethnia $select_ethnia
    --phenoFile $phenoFile
    --k $k
    --maxiter $maxiter
    --topk $topk
    --sigma $sigma
    --window $window
    --shift $shift
    --r2 $r2
    --trait_name $trait_name
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $pca_sos \
    --to-script $pca_sbatch \
    --args "$pca_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   /home/dc2325/project/UKBB_GWAS_dev/output/2021-02-22_flashpca_asian.sbatch
INFO: Workflow farnam (ID=wd32c86713853144c) is executed successfully with 1 completed step.



In [39]:
bfile=$UKBB_PATH/MWE/genotypes21_22.bed
genoFile=$UKBB_PATH/MWE/burden/ukb23155_c2*_b0_v1.plink.exome.filtered.bed
keep_samples=$UKBB_PATH/MWE/burden/unrelated_ind_burden.txt
phenoFile=$UKBB_PATH/MWE/burden/phenotype_burden.txt
famFile=$UKBB_PATH/MWE/burden/ukb23155_c21_b0_v1.plink.exome.filtered.fam
kinship=0.05
maf_filter=0.01
geno_filter=0.1
mind_filter=0.2 
hwe_filter=5e-08 
numThreads=2

sos dryrun ~/project/UKBB_GWAS_dev/workflow/PCA.ipynb filter\
    --cwd $pca_dir \
    --bfile $bfile \
    --genoFile $genoFile \
    --keep_samples $keep_samples \
    --kinship $kinship \
    --phenoFile $phenoFile \
    --famFile $famFile \
    --window $window \
    --shift $shift \
    --r2 $r2 \
    --maf_filter $maf_filter\
    --geno_filter $geno_filter\
    --mind_filter $mind_filter \
    --hwe_filter $hwe_filter \
    --numThreads $numThreads \
    --job_size $job_size \
    --container_lmm $container_lmm

INFO: Checking king: Inference of relationships in the sample to remove closely related individuals
HINT: singularity exec  /gpfs/gibbs/pi/dewan/data/UKBiobank/lmm.sif /bin/bash /gpfs/ysm/project/dewan/dc2325/UKBB_GWAS_dev/analysis/cluster_scripts/tmpo8bsz_hf/singularity_run_226211.sh
plink2 \
  --bfile /gpfs/gibbs/pi/dewan/data/UKBiobank/MWE/genotypes21_22 \
  --make-king-table \
  --threads 2 \
  --out /home/dc2325/scratch60/pca/genotypes21_22 


HINT: singularity exec  /gpfs/gibbs/pi/dewan/data/UKBiobank/lmm.sif Rscript /gpfs/ysm/project/dewan/dc2325/UKBB_GWAS_dev/analysis/cluster_scripts/tmp2itnm3gk/singularity_run_226211.R
library(dplyr)
kin0 <- read.table('/home/dc2325/scratch60/pca/genotypes21_22.kin0', header=F)
colnames(kin0) <- c("FID1","ID1","FID2","ID2","NSNP","HETHET","IBS0","KINSHIP")
rel <- kin0 %>%
    filter(KINSHIP >= 0.05)
cat("There are", nrow(rel),"related individuals using a kinship threshold of 0.05\n")
IID <- sort(unique(unlist(rel[, c("ID1", "ID2")])))
df <- da

In [ ]:
tpl_file=../farnam.yml
pca_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/pca_exomes
famFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb28374_exomedata/exome_data_OCT2020/ukb23155_s200631.fam
bedfiles=`echo /gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb28374_exomedata/exome_data_OCT2020/ukb23155_c{1..22}_b0_v1.bed`
bimfiles=`echo /gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb28374_exomedata/exome_data_OCT2020/UKBexomeOQFE_chr{1..22}.bim`
database=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/pleiotropy_R01/ukb42495_updatedJune2020/ukb42495.tab
# Container
container_lmm=/gpfs/gibbs/pi/dewan/data/UKBiobank/lmm.sif
# Pipeline
pca_sos=~/project/UKBB_GWAS_dev/PCA.ipynb
# Name of bash script
pca_sbatch=../output/$(date +"%Y-%m-%d")_pca_white.sbatch
numThreads=1
job_size=1
#PCA variables
k=10
maxiter=5
topk=10
sigma=6
window=50
shift=5
r2=0.5
stand="binom2"
maf_filter=0.01
geno_filter=0.01
mind_filter=0.02

sos run ~/project/UKBB_GWAS_dev/PCA.ipynb smartpca \
    --cwd $pca_dir \
    --bedfiles $bedfiles \
    --bimfiles $bimfiles \
    --famFile $famFile \
    --database $database \
    --k $k \
    --stand $stand \
    --maxiter $maxiter \
    --topk $topk \
    --sigma $sigma \
    --window $window \
    --shift $shift \
    --r2 $r2 \
    --maf_filter $maf_filter\
    --geno_filter $geno_filter\
    --mind_filter $mind_filter \
    --numThreads $numThreads \
    --job_size $job_size \
    --container_lmm $container_lmm \
    -s build

In [ ]:
    smartpca.perl \
    -i example.geno \
    -a example.snp \
    -b example.ind \
    -k 2 \
    -o example.pca \
    -p example.plot \
    -e example.eval \
    -l example.log \
    -m 5 \
    -t 2 \
    -s 6.0

In [ ]:
par.PACKEDPED.EIGENSTRAT
genotypename:    ukb23155_s200631.filtered.merged.bed
snpname:         ukb23155_s200631.filtered.merged.bim
indivname:       ukb23155_s200631.filtered.merged.fam
outputformat:    EIGENSTRAT
genotypeoutname: ukb23155_s200631.filtered.merged.eigenstratgeno
snpoutname:      ukb23155_s200631.filtered.merged.snp
indivoutname:    ukb23155_s200631.filtered.merged.ind

In [ ]:
#!/bin/bash
#SBATCH --partition general
#SBATCH --nodes 1
#SBATCH --ntasks-per-node 1
#SBATCH --cpus-per-task 1
#SBATCH --mem 60G
#SBATCH --time 5-0:00:00
#SBATCH --job-name ../output/2020-12-01_pca_white
#SBATCH --output ../output/2020-12-01_pca_white-%J.out
#SBATCH --error ../output/2020-12-01_pca_white-%J.log
module load EIGENSOFT/7.2.1-foss-2018b
smartpca.perl -i ukb23155_s200631.filtered.merged.bed -a ukb23155_s200631.filtered.merged.pedsnp -b ukb23155_s200631.filtered.merged.pedind -o ukb23155_s200631.filtered.merged.pca -p ukb23155_s200631.filtered.merged.plot -e ukb23155_s200631.filtered.eval -l ukb23155_s200631.filtered.merged.log

# Running plink missing pipeline

In [36]:
tpl_file=../farnam.yml
pca_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/pca_exomes
famFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb28374_exomedata/exome_data_OCT2020/ukb23155_s200631.fam
bedfiles=`echo /gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb28374_exomedata/exome_data_OCT2020/ukb23155_c{1..22}_b0_v1.bed`
bimfiles=`echo /gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb28374_exomedata/exome_data_OCT2020/UKBexomeOQFE_chr{1..22}.bim`
# Container
container_lmm=/gpfs/gibbs/pi/dewan/data/UKBiobank/lmm.sif
container_marp=/gpfs/gibbs/pi/dewan/data/UKBiobank/marp.sif
# Pipeline
plink_sos=~/project/UKBB_GWAS_dev/workflow/plink_missing.ipynb
# Name of bash script
pca_sbatch=../output/$(date +"%Y-%m-%d")_plink_miss.sbatch
numThreads=1
job_size=1


sos run ~/project/UKBB_GWAS_dev/workflow/plink_missing.ipynb missing\
    --cwd $pca_dir \
    --bedfiles $bedfiles \
    --bimfiles $bimfiles \
    --famFile $famFile \
    --numThreads $numThreads \
    --job_size $job_size \
    --container_lmm $container_lmm \
    --container_marp $container_marp \
    -s build

INFO: Running missing_1: Genotype and sample missingness for exome files
INFO: Step missing_1 (index=0) is ignored with signature constructed
INFO: Step missing_1 (index=1) is ignored with signature constructed
INFO: Step missing_1 (index=2) is ignored with signature constructed
INFO: Step missing_1 (index=3) is ignored with signature constructed
INFO: Step missing_1 (index=4) is ignored with signature constructed
INFO: Step missing_1 (index=5) is ignored with signature constructed
INFO: Step missing_1 (index=6) is ignored with signature constructed
INFO: Step missing_1 (index=7) is ignored with signature constructed
INFO: Step missing_1 (index=8) is ignored with signature constructed
INFO: Step missing_1 (index=9) is ignored with signature constructed
INFO: Step missing_1 (index=10) is ignored with signature constructed
INFO: Step missing_1 (index=11) is ignored with signature constructed
INFO: Step missing_1 (index=12) is ignored with signature constructed
INFO: Step missing_1 (index

## Extracting individuals for a particular snp plink

In [37]:
tpl_file=../farnam.yml
pca_dir=/home/dc2325/scratch60/plink_extract
famFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb28374_exomedata/exome_data_OCT2020/ukb23155_s200631.fam
bedfiles=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb28374_exomedata/exome_data_OCT2020/ukb23155_c12_b0_v1.bed
bimfiles=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb28374_exomedata/exome_data_OCT2020/UKBexomeOQFE_chr12.bim
snp_list=/home/dc2325/scratch60/plink_extract/snp.txt
# Container
container_lmm=/gpfs/gibbs/pi/dewan/data/UKBiobank/lmm.sif
container_marp=/gpfs/gibbs/pi/dewan/data/UKBiobank/marp.sif
# Pipeline
plink_sos=~/project/UKBB_GWAS_dev/plink_extract.ipynb
# Name of bash script
pca_sbatch=../output/$(date +"%Y-%m-%d")_plink_miss.sbatch
numThreads=1
job_size=1


sos run ~/project/UKBB_GWAS_dev/plink_extract.ipynb  \
    --cwd $pca_dir \
    --bedfiles $bedfiles \
    --bimfiles $bimfiles \
    --famFile $famFile \
    --snp_list $snp_list \
    --numThreads $numThreads \
    --job_size $job_size \
    --container_lmm $container_lmm \
    --container_marp $container_marp

ERROR: Failed to locate /home/dc2325/project/UKBB_GWAS_dev/plink_extract.ipynb.sos



## Regenie burden

In [23]:
genoFile=`echo /gpfs/gibbs/pi/dewan/data/UKBiobank/MWE/burden/ukb23155_c{21..22}_b0_v1.plink.exome.filtered.bed`
sos dryrun ~/project/bioworkflows/GWAS/LMM.ipynb regenie_burden \
    --cwd output \
    --bfile genotypes21_22.bed \
    --genoFile $genoFile \
    --sampleFile \
    --phenoFile burden/phenotype_burden.txt\
    --phenoCol ASTHMA\
    --covarCol SEX \
    --qCovarCol AGE \
    --numThreads 8 \
    --bsize 10 \
    --anno_file burden/annotation_file.txt\
    --set_list burden/set_list_file.txt \
    --mask_file burden/mask_file.txt \
    --keep_gene burden/keep_file.txt\
    --aaf_bins 0.05 \
    --trait bt \
    --build_mask max \
    --container_lmm /gpfs/gibbs/pi/dewan/data/UKBiobank/lmm.sif

INFO: Checking regenie_burden: Run regenie for burden tests
HINT: singularity exec  /gpfs/gibbs/pi/dewan/data/UKBiobank/lmm.sif /bin/bash /gpfs/ysm/project/dewan/dc2325/UKBB_GWAS_dev/analysis/tmph24c8d72/singularity_run_193934.sh
set -e
regenie \
  --step 2 \
  --bed /gpfs/gibbs/pi/dewan/data/UKBiobank/MWE/burden/ukb23155_c21_b0_v1.plink.exome.filtered \
  --phenoFile output/phenotype_burden.regenie_phenotype \
  --covarFile output/phenotype_burden.regenie_covar \
  --phenoColList ASTHMA \
  --bt
  --firth --approx \
  --pred output/phenotype_burden_ASTHMA.regenie_pred.list \
  --anno-file burden/annotation_file.txt \
  --set-list burden/set_list_file.txt \
  --extract-sets burden/keep_file.txt\
  --mask-def burden/mask_file.txt \
  --aaf-bins 0.05 \
  --write-mask \
  --build-mask \
  --bsize 10 \
  --check-burden-files \
  --gz \
  --out  output/cache/ukb23155_c21_b0_v1.plink.exome.filtered.burden


INFO: regenie_burden (index=0) is completed.
HINT: singularity exec  /gpfs/gibbs/pi/d